# Finding Swiss municipalities which names are also English Words

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from PyDictionary import PyDictionary
from io import StringIO
import sys
import os.path
from geopy.geocoders import Nominatim
import geopandas
from shapely.geometry import Point

In [3]:
class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout

In [4]:
def isEnglishWord(s):
    with Capturing() as output:
        dictionary.meaning(s)
    return output == []

## Matching municipalities with an english dictionary

In [5]:
df = pd.read_csv("./towns.csv")
dictionary = PyDictionary()

In [6]:
munis_english_words_path = 'municipalities_english_words.txt'
if(not os.path.exists(munis_english_words_path)):
    print("Computing municipalities...")
    municipalities_english_words = []
    for index, row in df.iterrows():
        municipality = row['Municipality']
        if(isEnglishWord(municipality)):
            municipalities_english_words.append(municipality)

    with open(munis_english_words_path, 'w') as filehandle:
        for muni in municipalities_english_words:
            filehandle.write('%s\n' % muni)
else:
    print("Retrieved computed municipalities from file")
    municipalities_english_words = []
    with open(munis_english_words_path, 'r') as filehandle:
        for line in filehandle:
            # remove linebreak which is the last character of the string
            currentPlace = line[:-1]
            municipalities_english_words.append(currentPlace)

Retrieved computed municipalities from file


## Handpicking which entries make sense
I chose to remove Units and proper nouns from the selected list, as they don't typically correspond to what we think about when thinking about english words

In [7]:
filtered_munis_path = "filtered_munis.txt"
if(not os.path.exists(filtered_munis_path)):
    #Filtering the results
    munis_deleting_decision = []

    for muni in municipalities_english_words:
        print(muni)
        print(dictionary.meaning(muni))
        keep = input("Keep?")
        munis_deleting_decision.append((muni, keep))

    filtered_muni_list = []
    for muni in munis_deleting_decision:
        if muni[1] == 'y': # y = keep
            filtered_muni_list.append(muni[0])

    with open(filtered_munis_path) as filehandle:
            for muni in filtered_muni_list:
                filehandle.write('%s\n' % muni)
else:
    print("Retrieved computed municipalities from file")
    filtered_muni_list = []
    with open(filtered_munis_path, 'r') as filehandle:
        for line in filehandle:
            # remove linebreak which is the last character of the string
            currentPlace = line[:-1]
            filtered_muni_list.append(currentPlace)

Retrieved computed municipalities from file


## Display data on map

In [8]:
#building a list of coordinates for the filtered entries
geolocator = Nominatim(user_agent="Finding_Swiss_Towns_With_English_names")
locations = pd.DataFrame([], columns = ['longitude', 'latitude'])

for i in range(len(filtered_muni_list)):
    muni = filtered_muni_list[i]
    location = geolocator.geocode(muni + " Switzerland", timeout=10)
    locations.loc[i] = [location.longitude, location.latitude]

In [9]:
locations

,longitude,latitude
0,8.627116,47.473176
1,7.429058,47.167113
2,7.677127,46.848498
3,7.337079,47.239899
4,7.081828,47.113012
5,7.257291,47.119211
6,9.310289,47.435172
7,8.391109,47.115344
8,8.275348,46.900811
9,6.821521,46.619839
